<a href="https://www.kaggle.com/code/iniyansel/market-basket-association-analysis?scriptVersionId=194692365" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/market-basket-analysis/Assignment-1_Data.xlsx
/kaggle/input/market-basket-analysis/Assignment-1_Data.csv


# Importing necessary libraries

In [2]:
import pandas as pd #for processing
from mlxtend.frequent_patterns import apriori, association_rules #association analysis

# Reading the data

In [3]:
df = pd.read_csv("/kaggle/input/market-basket-analysis/Assignment-1_Data.csv",sep= ";", low_memory = False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [5]:
df.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


# Data cleaning

In [6]:
df["Itemname"] = df["Itemname"].str.strip()

In [7]:
df["BillNo"] = df["BillNo"].astype(str)

In [8]:
df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y %H:%M")

In [9]:
df["Price"] = df["Price"].str.replace(",",".")

In [10]:
df["Price"] = pd.to_numeric(df["Price"])

In [11]:
df["CustomerID"] = df["CustomerID"].astype(str).str.replace(".0","")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   BillNo      522064 non-null  object        
 1   Itemname    520609 non-null  object        
 2   Quantity    522064 non-null  int64         
 3   Date        522064 non-null  datetime64[ns]
 4   Price       522064 non-null  float64       
 5   CustomerID  522064 non-null  object        
 6   Country     522064 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 27.9+ MB


# Creating a transaction DataFrame

In [13]:
baskets = (df.groupby(["BillNo","Itemname"])["Quantity"].sum().unstack()).fillna(0)

In [14]:
baskets

Itemname,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A563185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def to_bool(x):
    if x <= 0:
        return False
    if x >= 1:
        return True

In [16]:
basketbool = baskets.apply(lambda col: col.map(to_bool))

In [17]:
basketbool

Itemname,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536366,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536367,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536368,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536369,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581586,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581587,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
A563185,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Association Analysis

In [18]:
frequent_combos = apriori(basketbool, min_support=0.02, use_colnames=True)

In [19]:
frequent_combos

,support,itemsets
0,0.022615,(3 STRIPEY MICE FELTCRAFT)
1,0.022961,(4 TRADITIONAL SPINNING TOPS)
2,0.046615,(6 RIBBONS RUSTIC CHARM)
3,0.020635,(60 CAKE CASES DOLLY GIRL DESIGN)
4,0.029691,(60 CAKE CASES VINTAGE CHRISTMAS)
...,...,...
353,0.024198,"(REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU..."
354,0.020042,"(WHITE HANGING HEART T-LIGHT HOLDER, WOODEN PI..."
355,0.026524,"(WOODEN PICTURE FRAME WHITE FINISH, WOODEN FRA..."
356,0.025485,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY..."


In [20]:
rules = association_rules(frequent_combos, metric="lift", min_threshold=1)

In [21]:
rules.sort_values(by="lift", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
154,"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",(PINK REGENCY TEACUP AND SAUCER),0.036174,0.036421,0.025485,0.704514,19.343514,0.024167,3.261000,0.983894
155,(PINK REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",0.036421,0.036174,0.025485,0.699728,19.343514,0.024167,3.209847,0.984147
152,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.028207,0.048248,0.025485,0.903509,18.726262,0.024124,9.863609,0.974074
157,(GREEN REGENCY TEACUP AND SAUCER),"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",0.048248,0.028207,0.025485,0.528205,18.726262,0.024124,2.059779,0.994586
25,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.048248,0.036421,0.029939,0.620513,17.037124,0.028181,2.539160,0.989023
...,...,...,...,...,...,...,...,...,...,...
149,(WOODEN PICTURE FRAME WHITE FINISH),(WHITE HANGING HEART T-LIGHT HOLDER),0.054038,0.108967,0.020042,0.370879,3.403599,0.014153,1.416315,0.746535
131,(WHITE HANGING HEART T-LIGHT HOLDER),(NATURAL SLATE HEART CHALKBOARD),0.108967,0.060966,0.020338,0.186649,3.061520,0.013695,1.154524,0.755712
130,(NATURAL SLATE HEART CHALKBOARD),(WHITE HANGING HEART T-LIGHT HOLDER),0.060966,0.108967,0.020338,0.333604,3.061520,0.013695,1.337093,0.717083
80,(WHITE HANGING HEART T-LIGHT HOLDER),(JUMBO BAG RED RETROSPOT),0.108967,0.102138,0.021873,0.200727,1.965254,0.010743,1.123348,0.551225
